In [1]:
segment_plain = {'CUARTO 401 - 3':'CUARTO', 'CUARTO 402 - 3':'CUARTO', 'CUARTO 403 - 3':'CUARTO',
       'PRIMERO 101 - 3':'PRIMERO', 'PRIMERO 102 - 3':'PRIMERO', 'PRIMERO 103 - 3':'PRIMERO',
       'QUINTO 501 - 3':'QUINTO', 'QUINTO 502 - 3':'QUINTO', 'QUINTO 503 - 3':'QUINTO',
       'SEGUNDO 201 - 3':'SEGUNDO', 'SEGUNDO 202 - 3':'SEGUNDO', 'SEGUNDO 203 - 3':'SEGUNDO',
       'TERCERO 301 - 3':'TERCERO', 'TERCERO 302 - 3':'TERCERO', 'TERCERO 303 - 3':'TERCERO',
       'CUARTO 401 - 4':'CUARTO', 'CUARTO 402 - 4':'CUARTO', 'PRIMERO 101 - 4':'PRIMERO',
       'PRIMERO 102 - 4':'PRIMERO', 'QUINTO 501 - 4':'QUINTO', 'QUINTO 502 - 4':'QUINTO',
       'SEGUNDO 201 - 4':'SEGUNDO', 'SEGUNDO 202 - 4':'SEGUNDO', 'SEGUNDO 203 - 4':'SEGUNDO',
       'TERCERO 301 - 4':'TERCERO', 'TERCERO 302 - 4':'TERCERO', 'TERCERO 303 - 4':'TERCERO'}

In [2]:
maping =  {'subject_1-nota_y':"Ciencias naturales y educación ambiental",
			'subject_2-nota':'Ciencias sociales, historia, geografía, constitución política y democracia',
			'subject_3-nota':"Educación artística y Cultural",
			'subject_4-nota':"Educación ética y en valores humanos",
			'subject_5-nota':"Educación física,  recreación y deportes",
			'subject_6-nota':"Educación religiosa. ",
			'subject_7-nota':"Humanidades, lengua castellana",
			'subject_8-nota':"Idioma extranjero  Inglés",
			'subject_9-nota':"Matemáticas",
			'subject_10-nota':"Sensibilización Pedagógica",
			'subject_11-nota':"Tecnología e informática"}

In [3]:
maping_primaria_nuevo =  {'subject_1-nota_y':"Ciencias naturales",
			'subject_2-nota':'Ciencias sociales',
			'subject_3-nota':"Educación artística",
			'subject_4-nota':"Educación ética",
			'subject_5-nota':"Educación física",
			'subject_6-nota':"Educación religiosa. ",
			'subject_7-nota':"Lengua castellana",
			'subject_8-nota':"Idioma extranjero",
			'subject_9-nota':"Matemáticas",
			'subject_10-nota':"Sensibilización Pedagógica",
			'subject_11-nota':"Tecnología e informática",
            'subject_12-nota':"Humanities"}

In [66]:
import pandas as pd
import datetime

def data():
    df = pd.read_csv('data.csv')
    filtro_notas = {'AB':'2', 'AA':'4', 'NA':'1', 'AM':'3'}
    df = df.replace(filtro_notas)
    df['fecha'] = pd.to_datetime(df['fecha'])
    df['nombre'] = df.nombre.str.strip()
    df['segment'] = df.segment.map(segment_plain)
    df_2 = df[df.fecha=='2020-09-01'].reset_index().drop('index',axis=1)
    n=0
    for i in df[df.fecha=='2020-12-01'].index:
        for j in range(4,17):
            df.iloc[i,j:j+1] = df_2.iloc[n,j:j+1]
        n=n+1
    df['mes'] = df.fecha.dt.month
    df=df.fillna(method='bfill')
    #df['periodo'] = df.periodo.map({'p1':'1','p2':'2','p3':'3','p4':'4'})
    #df['periodo']= df['periodo'].astype(int)
    df['subject_12-nota'] = (df['subject_4-nota']+df['subject_6-nota'])/2
    df['subject_12-nota'] = df['subject_12-nota'].astype(int)
    for i in df[(df.mes==12)&(df.periodo=='p3')].index:
        df.iloc[i,4] = 'p4'
    return df

def periodos(fecha,df):
    '''
    Filtra la información de los tres periodos anteriores a la fecha escogida. Luego asigna como p4
    al periodo de la fecha escogida, este será el target, y modifica las asignaciones de los periodos
    para que vaya hacia atras como periodo p2, luego el p2 y por ultimo el p1. Con esto
    garantizamos la estandarización, de p1 a p3 como datos de entrada para predecir p4.
    PARAMETROS: Fecha en la cual se quiere realizar que vamos a utilizar como target para entrenar el modelo
    RETORNA: DataFrame filtrado con los periodos en orden.
    '''
    fecha_prediccion = pd.to_datetime(fecha)# aca se escoge una fecha en particular
    df = df[df.nombre.isin(df[df.fecha == fecha_prediccion].nombre)]
    fecha_inicio_prediccion = fecha_prediccion - datetime.timedelta(weeks=52) # se obtiene la fecha para incluir los tres periodos anteriores a la fecha elegida para la predicción
    df1 = df[(df.fecha>= fecha_inicio_prediccion) & (df.fecha<=fecha_prediccion)]# se filtra entre las fechas escogidas
    if fecha_prediccion.month == 3:
        df_final = df1.replace({'p1':'p4','p4':'p3','p3':'p2','p2':'p1'})
    elif fecha_prediccion.month == 6:
        df_final = df1.replace({'p2':'p4','p1':'p3','p4':'p2','p3':'p1'})
    elif fecha_prediccion.month == 9:
        df_final = df1.replace({'p3':'p4','p2':'p3','p1':'p2','p4':'p1'})
    elif fecha_prediccion.month == 12:
        df_final = df1.replace({'p3':'p3','p2':'p2','p1':'p1','p4':'p4'})
    return df_final  

def finalDataFrame(df,fecha):
    fecha_prediccion = pd.to_datetime(fecha)
    dict_segment = df[df.fecha == fecha_prediccion][['nombre','segment']].set_index('nombre').T.to_dict('list')
    df2=pd.pivot_table(df,index='nombre',columns='periodo',values=['subject_1-nota_y',
       'subject_2-nota', 'subject_3-nota', 'subject_4-nota', 'subject_5-nota',
       'subject_6-nota', 'subject_7-nota', 'subject_8-nota', 'subject_9-nota',
       'subject_10-nota', 'subject_11-nota'],dropna=False)
    df2.columns = df2.columns.map('_'.join)
    df2['fecha'] = fecha_prediccion
    df2['segment'] = 'NAN'
    df2 = df2.reset_index()   
    names = df2.nombre.unique()
    
    for i in range(len(df2.nombre)):
        df2.segment[i] = dict_segment[df2.nombre[i]][0]
        
    return df2

df_final = pd.DataFrame(columns=['nombre', 'subject_1-nota_y_p1', 'subject_1-nota_y_p2',
       'subject_1-nota_y_p3', 'subject_1-nota_y_p4', 'subject_2-nota_p1',
       'subject_2-nota_p2', 'subject_2-nota_p3', 'subject_2-nota_p4',
       'subject_3-nota_p1', 'subject_3-nota_p2', 'subject_3-nota_p3',
       'subject_3-nota_p4', 'subject_4-nota_p1', 'subject_4-nota_p2',
       'subject_4-nota_p3', 'subject_4-nota_p4', 'subject_5-nota_p1',
       'subject_5-nota_p2', 'subject_5-nota_p3', 'subject_5-nota_p4',
       'subject_6-nota_p1', 'subject_6-nota_p2', 'subject_6-nota_p3',
       'subject_6-nota_p4', 'subject_7-nota_p1', 'subject_7-nota_p2',
       'subject_7-nota_p3', 'subject_7-nota_p4', 'subject_8-nota_p1',
       'subject_8-nota_p2', 'subject_8-nota_p3', 'subject_8-nota_p4',
       'subject_9-nota_p1', 'subject_9-nota_p2', 'subject_9-nota_p3',
       'subject_9-nota_p4', 'subject_10-nota_p1', 'subject_10-nota_p2',
       'subject_10-nota_p3', 'subject_10-nota_p4', 'subject_11-nota_p1',
       'subject_11-nota_p2', 'subject_11-nota_p3', 'subject_11-nota_p4',
       'fecha'])
lista_fecha_periodos = ['2021-03-01','2021-06-01','2021-09-01','2021-12-01','2020-03-01','2020-06-01','2020-09-01',
                        '2020-12-01','2019-03-01','2019-06-01','2019-09-01','2019-12-01','2018-12-01']
df = data()
for fecha in lista_fecha_periodos:
    df_periodo=periodos(fecha,df)
    df_pivot = finalDataFrame(df_periodo,fecha)
    df_final = pd.concat([df_final, df_pivot])
df_final

/var/folders/k7/ncn3z0qs4h7f50pl2pw0l20r0000gp/T/ipykernel_22547/2455974752.py:64: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df2.segment[i] = dict_segment[df2.nombre[i]][0]


,nombre,subject_1-nota_y_p1,subject_1-nota_y_p2,subject_1-nota_y_p3,subject_1-nota_y_p4,subject_2-nota_p1,subject_2-nota_p2,subject_2-nota_p3,subject_2-nota_p4,subject_3-nota_p1,...,subject_10-nota_p1,subject_10-nota_p2,subject_10-nota_p3,subject_10-nota_p4,subject_11-nota_p1,subject_11-nota_p2,subject_11-nota_p3,subject_11-nota_p4,fecha,segment
0,ACEVEDO MOLANO MATIAS,3.0,4.0,4.0,4.0,3.0,4.0,4.0,4.0,3.0,...,3.0,3.0,3.0,3.0,3.0,3.0,3.0,3.0,2021-03-01,SEGUNDO
1,ACEVEDO MOLANO THOMAS,3.0,4.0,4.0,4.0,3.0,3.0,3.0,4.0,3.0,...,3.0,3.0,3.0,3.0,3.0,3.0,3.0,3.0,2021-03-01,QUINTO
2,ACOSTA BASTO SARA SOFIA,3.0,3.0,3.0,3.0,3.0,3.0,3.0,3.0,3.0,...,3.0,3.0,3.0,3.0,3.0,3.0,3.0,3.0,2021-03-01,TERCERO
3,AGUDELO ARANGO JUAN MANUEL,3.0,3.0,3.0,3.0,3.0,3.0,3.0,3.0,3.0,...,3.0,3.0,3.0,3.0,3.0,3.0,3.0,3.0,2021-03-01,SEGUNDO
4,AGUDELO CARDONA GUADALUPE,3.0,3.0,3.0,3.0,4.0,3.0,3.0,3.0,4.0,...,3.0,3.0,3.0,4.0,4.0,4.0,4.0,4.0,2021-03-01,TERCERO
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1050,ZAPATA ZAPATA JUAN FELIPE,4.0,4.0,4.0,4.0,4.0,3.0,4.0,3.0,4.0,...,4.0,4.0,4.0,4.0,4.0,4.0,4.0,4.0,2018-12-01,TERCERO
1051,ZULUAGA ACEVEDO NATALIA,4.0,4.0,4.0,3.0,4.0,4.0,2.0,4.0,3.0,...,4.0,4.0,3.0,4.0,4.0,3.0,2.0,3.0,2018-12-01,PRIMERO
1052,ZULUAGA DUQUE MANUELA,4.0,2.0,2.0,4.0,4.0,4.0,2.0,2.0,3.0,...,3.0,3.0,4.0,2.0,3.0,4.0,4.0,4.0,2018-12-01,TERCERO
1053,ZULUAGA GAVIRIA MELANY,3.0,3.0,3.0,3.0,3.0,3.0,3.0,4.0,3.0,...,3.0,3.0,3.0,3.0,3.0,3.0,3.0,3.0,2018-12-01,CUARTO


In [5]:
df_final.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 13979 entries, 0 to 1054
Data columns (total 47 columns):
 #   Column               Non-Null Count  Dtype         
---  ------               --------------  -----         
 0   nombre               13979 non-null  object        
 1   subject_1-nota_y_p1  11681 non-null  float64       
 2   subject_1-nota_y_p2  12447 non-null  float64       
 3   subject_1-nota_y_p3  13213 non-null  float64       
 4   subject_1-nota_y_p4  13979 non-null  float64       
 5   subject_2-nota_p1    11681 non-null  float64       
 6   subject_2-nota_p2    12447 non-null  float64       
 7   subject_2-nota_p3    13213 non-null  float64       
 8   subject_2-nota_p4    13979 non-null  float64       
 9   subject_3-nota_p1    11681 non-null  float64       
 10  subject_3-nota_p2    12447 non-null  float64       
 11  subject_3-nota_p3    13213 non-null  float64       
 12  subject_3-nota_p4    13979 non-null  float64       
 13  subject_4-nota_p1    11681 non-n

In [67]:
df_final = df_final.dropna()
df_final = df_final.reset_index(drop=True)

In [68]:
df_final.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 11681 entries, 0 to 11680
Data columns (total 47 columns):
 #   Column               Non-Null Count  Dtype         
---  ------               --------------  -----         
 0   nombre               11681 non-null  object        
 1   subject_1-nota_y_p1  11681 non-null  float64       
 2   subject_1-nota_y_p2  11681 non-null  float64       
 3   subject_1-nota_y_p3  11681 non-null  float64       
 4   subject_1-nota_y_p4  11681 non-null  float64       
 5   subject_2-nota_p1    11681 non-null  float64       
 6   subject_2-nota_p2    11681 non-null  float64       
 7   subject_2-nota_p3    11681 non-null  float64       
 8   subject_2-nota_p4    11681 non-null  float64       
 9   subject_3-nota_p1    11681 non-null  float64       
 10  subject_3-nota_p2    11681 non-null  float64       
 11  subject_3-nota_p3    11681 non-null  float64       
 12  subject_3-nota_p4    11681 non-null  float64       
 13  subject_4-nota_p1    11681 non-

In [70]:
df_final.tail()

,nombre,subject_1-nota_y_p1,subject_1-nota_y_p2,subject_1-nota_y_p3,subject_1-nota_y_p4,subject_2-nota_p1,subject_2-nota_p2,subject_2-nota_p3,subject_2-nota_p4,subject_3-nota_p1,...,subject_10-nota_p1,subject_10-nota_p2,subject_10-nota_p3,subject_10-nota_p4,subject_11-nota_p1,subject_11-nota_p2,subject_11-nota_p3,subject_11-nota_p4,fecha,segment
11676,ZAPATA ZAPATA JUAN FELIPE,4.0,4.0,4.0,4.0,4.0,3.0,4.0,3.0,4.0,...,4.0,4.0,4.0,4.0,4.0,4.0,4.0,4.0,2018-12-01,TERCERO
11677,ZULUAGA ACEVEDO NATALIA,4.0,4.0,4.0,3.0,4.0,4.0,2.0,4.0,3.0,...,4.0,4.0,3.0,4.0,4.0,3.0,2.0,3.0,2018-12-01,PRIMERO
11678,ZULUAGA DUQUE MANUELA,4.0,2.0,2.0,4.0,4.0,4.0,2.0,2.0,3.0,...,3.0,3.0,4.0,2.0,3.0,4.0,4.0,4.0,2018-12-01,TERCERO
11679,ZULUAGA GAVIRIA MELANY,3.0,3.0,3.0,3.0,3.0,3.0,3.0,4.0,3.0,...,3.0,3.0,3.0,3.0,3.0,3.0,3.0,3.0,2018-12-01,CUARTO
11680,ZULUAGA ORTEGA LAUREN SOFIA,4.0,4.0,3.0,3.0,4.0,4.0,4.0,4.0,4.0,...,3.0,3.0,3.0,3.0,4.0,3.0,3.0,3.0,2018-12-01,SEGUNDO


In [20]:
df_final_copy = df_final.head(50).copy()
df_final_copy

,nombre,subject_1-nota_y_p1,subject_1-nota_y_p2,subject_1-nota_y_p3,subject_1-nota_y_p4,subject_2-nota_p1,subject_2-nota_p2,subject_2-nota_p3,subject_2-nota_p4,subject_3-nota_p1,...,subject_10-nota_p1,subject_10-nota_p2,subject_10-nota_p3,subject_10-nota_p4,subject_11-nota_p1,subject_11-nota_p2,subject_11-nota_p3,subject_11-nota_p4,fecha,segment
0,ACEVEDO MOLANO MATIAS,3.0,4.0,4.0,4.0,3.0,4.0,4.0,4.0,3.0,...,3.0,3.0,3.0,3.0,3.0,3.0,3.0,3.0,2021-03-01,SEGUNDO
1,ACEVEDO MOLANO THOMAS,3.0,4.0,4.0,4.0,3.0,3.0,3.0,4.0,3.0,...,3.0,3.0,3.0,3.0,3.0,3.0,3.0,3.0,2021-03-01,QUINTO
2,ACOSTA BASTO SARA SOFIA,3.0,3.0,3.0,3.0,3.0,3.0,3.0,3.0,3.0,...,3.0,3.0,3.0,3.0,3.0,3.0,3.0,3.0,2021-03-01,TERCERO
3,AGUDELO ARANGO JUAN MANUEL,3.0,3.0,3.0,3.0,3.0,3.0,3.0,3.0,3.0,...,3.0,3.0,3.0,3.0,3.0,3.0,3.0,3.0,2021-03-01,SEGUNDO
4,AGUDELO CARDONA GUADALUPE,3.0,3.0,3.0,3.0,4.0,3.0,3.0,3.0,4.0,...,3.0,3.0,3.0,4.0,4.0,4.0,4.0,4.0,2021-03-01,TERCERO
5,AGUDELO CARDONA ISABELLA,3.0,4.0,4.0,4.0,3.0,3.0,3.0,3.0,3.0,...,4.0,3.0,3.0,3.0,3.0,3.0,3.0,4.0,2021-03-01,QUINTO
7,AGUDELO CORRALES JUAN ALEJANDRO,3.0,4.0,4.0,4.0,3.0,4.0,4.0,4.0,4.0,...,3.0,4.0,4.0,3.0,3.0,4.0,4.0,4.0,2021-03-01,SEGUNDO
8,AGUDELO CORRALES JUAN PABLO,3.0,4.0,4.0,4.0,3.0,4.0,4.0,4.0,4.0,...,3.0,4.0,4.0,3.0,3.0,4.0,4.0,4.0,2021-03-01,SEGUNDO
9,AGUDELO CORREA MARIANA,4.0,4.0,4.0,4.0,4.0,4.0,4.0,4.0,4.0,...,3.0,3.0,3.0,3.0,3.0,3.0,3.0,4.0,2021-03-01,TERCERO
11,AGUDELO LONDOÑO JUAN JOSE,4.0,3.0,3.0,3.0,4.0,4.0,4.0,3.0,3.0,...,4.0,3.0,3.0,4.0,4.0,3.0,3.0,3.0,2021-03-01,QUINTO


In [10]:
def flagsFirst(df,nombre,fecha):
    df_labels = df[df.fecha==fecha]
    df_labels = df_labels[df_labels.nombre == nombre]
    quantiles = df_labels.groupby(['subject']).grade.quantile([0.3,0.5,0.8])
    materias = df_labels.subject.unique()
    lista = []
    for materia in materias:
        grade = df_labels[df_labels.subject==materia].grade.iloc[0]
        if grade>=quantiles[materia][0.8]:
            lista.append(4)
        elif (grade>=quantiles[materia][0.5]) and (grade<quantiles['Ciencias naturales'][0.8]):
            lista.append(3)
        elif (grade>=quantiles[materia][0.3]) and (grade<quantiles['Ciencias naturales'][0.5]):
            lista.append(2)
        else:
            lista.append(1)
    return(lista)

In [11]:
def evaluateFlagsFirst(lista):
    if lista.count(1)>=5:
        return('RedFlag')
    elif lista.count(1)>=4:
        return('OrangeFlag')
    elif lista.count(1)>=3:
        return('YellowFlag')
    elif lista.count(1)>=2:
        return('WhiteFlag')
    elif lista.count(4)>=1:
        return('GreenFlag')

In [12]:
def flagsSecond(df,fecha,nombre):
    diccionario = {'danger':0,'neutral':0,'positive':0}
    df_final = tableSubject(fecha,df)
    df_final['subject'] = df_final.subject.map(maping_primaria_nuevo)
    materias = df_final.subject.unique()
    df_final = df_final[df_final.nombre == nombre]
    for materia in materias:
        df_final2 = df_final[df_final['subject']==materia]
        highest = df_final2[df_final2.periodo!='p4'].grade.max()
        actual = df_final2[df_final2.periodo=='p4'].grade.iloc[0]
        diff = actual - highest
        if diff <= -2:
            diccionario['danger'] = diccionario['danger']+1
        elif (diff >= -1) & (diff < 2):
            diccionario['neutral'] = diccionario['neutral']+1
        elif diff == 2:
            diccionario['positive'] = diccionario['positive']+1
    return diccionario

In [13]:
def evaluateFlagsSecond(diccionario):
    if diccionario['danger']>=5:
        return('RedFlag')
    elif diccionario['danger']>=3:
        return('OrangeFlag')
    elif diccionario['danger']>=1:
        return('YellowFlag')
    elif diccionario['positive']>=5:
        return('GreenFlag')
    else:
        return('WhiteFlag')

In [14]:
df = data()
df_melt= df.melt(id_vars=['nombre', 'fecha', 'segment','periodo'], 
                               value_vars=['subject_1-nota_y','subject_2-nota', 'subject_3-nota', 
                                           'subject_5-nota','subject_12-nota', 
                                           'subject_7-nota', 'subject_8-nota', 'subject_9-nota','subject_10-nota', 
                                           'subject_11-nota'],value_name='grade',var_name='subject')
df_melt['subject'] = df_melt.subject.map(maping_primaria_nuevo)

In [15]:
def tableSubject(fecha,df):
    '''
    esta función retornara un dataframe que sera usado para graficar las notas de un 
    estudiante en específico para todas las materias
    '''
    df_periodo=periodos(fecha,df)
    df_nueva = df_periodo.melt(id_vars=['nombre', 'fecha', 'segment','periodo'], 
                               value_vars=['subject_1-nota_y','subject_2-nota', 'subject_3-nota', 
                                           'subject_5-nota','subject_12-nota', 
                                           'subject_7-nota', 'subject_8-nota', 'subject_9-nota','subject_10-nota', 
                                           'subject_11-nota'],value_name='grade',var_name='subject')
    return df_nueva

In [16]:
fecha = '2020-09-01'
nombre = 'GUEVARA ROJAS SAMUEL'
#first
lista = flagsFirst(df_melt,nombre,fecha)
firstLavel = evaluateFlagsFirst(lista)
#second
diccionario = flagsSecond(df,fecha,nombre)
secondLavel = evaluateFlagsSecond(diccionario)
print ('firstLavel: ', firstLavel)
print ('secondLavel: ', secondLavel)

firstLavel:  GreenFlag
secondLavel:  WhiteFlag


In [73]:
df_final_copy = df_final.head(1000).copy()

In [74]:
df_final_copy['firstLabel'] = 'NAN'
df_final_copy['secondLabel'] = 'NAN'
for i in df_final_copy.index:
    fecha = df_final_copy.fecha[i]
    nombre = df_final_copy.nombre[i]
    #first
    lista = flagsFirst(df_melt,nombre,fecha)
    df_final_copy['firstLabel'][i] = evaluateFlagsFirst(lista)
    #second
    diccionario = flagsSecond(df,fecha,nombre)
    df_final_copy['secondLabel'][i] = evaluateFlagsSecond(diccionario)
df_final_copy

/var/folders/k7/ncn3z0qs4h7f50pl2pw0l20r0000gp/T/ipykernel_22547/4214330145.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_final_copy['firstLavel'][i] = evaluateFlagsFirst(lista)
/var/folders/k7/ncn3z0qs4h7f50pl2pw0l20r0000gp/T/ipykernel_22547/4214330145.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_final_copy['secondLavel'][i] = evaluateFlagsSecond(diccionario)


,nombre,subject_1-nota_y_p1,subject_1-nota_y_p2,subject_1-nota_y_p3,subject_1-nota_y_p4,subject_2-nota_p1,subject_2-nota_p2,subject_2-nota_p3,subject_2-nota_p4,subject_3-nota_p1,...,subject_10-nota_p3,subject_10-nota_p4,subject_11-nota_p1,subject_11-nota_p2,subject_11-nota_p3,subject_11-nota_p4,fecha,segment,firstLavel,secondLavel
0,ACEVEDO MOLANO MATIAS,3.0,4.0,4.0,4.0,3.0,4.0,4.0,4.0,3.0,...,3.0,3.0,3.0,3.0,3.0,3.0,2021-03-01,SEGUNDO,GreenFlag,WhiteFlag
1,ACEVEDO MOLANO THOMAS,3.0,4.0,4.0,4.0,3.0,3.0,3.0,4.0,3.0,...,3.0,3.0,3.0,3.0,3.0,3.0,2021-03-01,QUINTO,GreenFlag,WhiteFlag
2,ACOSTA BASTO SARA SOFIA,3.0,3.0,3.0,3.0,3.0,3.0,3.0,3.0,3.0,...,3.0,3.0,3.0,3.0,3.0,3.0,2021-03-01,TERCERO,GreenFlag,WhiteFlag
3,AGUDELO ARANGO JUAN MANUEL,3.0,3.0,3.0,3.0,3.0,3.0,3.0,3.0,3.0,...,3.0,3.0,3.0,3.0,3.0,3.0,2021-03-01,SEGUNDO,GreenFlag,WhiteFlag
4,AGUDELO CARDONA GUADALUPE,3.0,3.0,3.0,3.0,4.0,3.0,3.0,3.0,4.0,...,3.0,4.0,4.0,4.0,4.0,4.0,2021-03-01,TERCERO,GreenFlag,WhiteFlag
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
995,CARVAJAL GIL ISABELLA,4.0,4.0,4.0,4.0,4.0,4.0,3.0,3.0,3.0,...,3.0,3.0,3.0,3.0,4.0,4.0,2021-06-01,TERCERO,GreenFlag,WhiteFlag
996,CASALLAS LOPEZ NICOLAS,3.0,3.0,3.0,3.0,3.0,3.0,4.0,3.0,3.0,...,3.0,3.0,3.0,3.0,3.0,3.0,2021-06-01,QUINTO,GreenFlag,WhiteFlag
997,CASALLAS SALGADO MATHIAS,4.0,4.0,2.0,4.0,4.0,4.0,4.0,4.0,4.0,...,3.0,4.0,4.0,4.0,4.0,4.0,2021-06-01,SEGUNDO,GreenFlag,WhiteFlag
998,CASAS GARCIA SARA,4.0,4.0,2.0,4.0,4.0,4.0,4.0,4.0,4.0,...,3.0,4.0,4.0,4.0,3.0,4.0,2021-06-01,QUINTO,GreenFlag,YellowFlag


In [78]:
df_final_copy.secondLavel.value_counts()

WhiteFlag     759
YellowFlag    173
OrangeFlag     41
RedFlag        19
GreenFlag       8
Name: secondLavel, dtype: int64

In [79]:
df_final_copy[df_final_copy.secondLavel == 'RedFlag']

,nombre,subject_1-nota_y_p1,subject_1-nota_y_p2,subject_1-nota_y_p3,subject_1-nota_y_p4,subject_2-nota_p1,subject_2-nota_p2,subject_2-nota_p3,subject_2-nota_p4,subject_3-nota_p1,...,subject_10-nota_p3,subject_10-nota_p4,subject_11-nota_p1,subject_11-nota_p2,subject_11-nota_p3,subject_11-nota_p4,fecha,segment,firstLavel,secondLavel
45,ARAQUE MARTINEZ JUAN STEVAN,4.0,4.0,4.0,1.0,4.0,4.0,4.0,2.0,4.0,...,4.0,2.0,4.0,4.0,4.0,3.0,2021-03-01,QUINTO,GreenFlag,RedFlag
133,CAMPUZANO ARIAS SARA,4.0,2.0,2.0,1.0,4.0,2.0,2.0,2.0,3.0,...,4.0,2.0,4.0,2.0,2.0,2.0,2021-03-01,TERCERO,GreenFlag,RedFlag
153,CARDOZO HENAO MARIA ANTONIA,2.0,2.0,2.0,1.0,2.0,2.0,2.0,1.0,4.0,...,4.0,1.0,2.0,4.0,4.0,1.0,2021-03-01,SEGUNDO,GreenFlag,RedFlag
181,CASTRILLON LOPEZ VALERIA,4.0,4.0,4.0,2.0,4.0,4.0,4.0,1.0,4.0,...,2.0,2.0,4.0,4.0,4.0,3.0,2021-03-01,QUINTO,GreenFlag,RedFlag
241,ESCOBAR MORALES ISABELLA,4.0,4.0,4.0,2.0,4.0,4.0,4.0,2.0,4.0,...,2.0,2.0,4.0,4.0,4.0,3.0,2021-03-01,QUINTO,GreenFlag,RedFlag
444,LOPEZ MARIN MARIANA,4.0,4.0,4.0,1.0,2.0,4.0,4.0,1.0,3.0,...,3.0,1.0,3.0,3.0,3.0,2.0,2021-03-01,QUINTO,GreenFlag,RedFlag
456,MARIN AMPUDIA LAURA SOFIA,4.0,4.0,4.0,2.0,4.0,4.0,4.0,2.0,3.0,...,3.0,3.0,3.0,3.0,3.0,3.0,2021-03-01,CUARTO,GreenFlag,RedFlag
528,OCAMPO QUINTERO SAMUEL,4.0,4.0,4.0,2.0,4.0,4.0,4.0,2.0,4.0,...,4.0,2.0,4.0,3.0,3.0,2.0,2021-03-01,TERCERO,GreenFlag,RedFlag
532,OROZCO MONTOYA MARIANTONIA,4.0,4.0,4.0,2.0,4.0,4.0,4.0,2.0,4.0,...,4.0,2.0,4.0,3.0,3.0,4.0,2021-03-01,TERCERO,GreenFlag,RedFlag
596,POSADA GONZALEZ SANTIAGO,4.0,2.0,2.0,2.0,2.0,2.0,2.0,4.0,4.0,...,4.0,2.0,4.0,4.0,4.0,2.0,2021-03-01,SEGUNDO,GreenFlag,RedFlag


In [54]:
df_final_copy = df_final.iloc[500:1000].copy()

In [55]:
df_final_copy['firstLabel'] = 'NAN'
df_final_copy['secondLabel'] = 'NAN'
for i in df_final_copy.index:
    fecha = df_final_copy.fecha[i]
    nombre = df_final_copy.nombre[i]
    #first
    lista = flagsFirst(df_melt,nombre,fecha)
    df_final_copy['firstLabel'][i] = evaluateFlagsFirst(lista)
    #second
    diccionario = flagsSecond(df,fecha,nombre)
    df_final_copy['secondLabel'][i] = evaluateFlagsSecond(diccionario)
df_final_copy

/var/folders/k7/ncn3z0qs4h7f50pl2pw0l20r0000gp/T/ipykernel_22547/4214330145.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_final_copy['firstLavel'][i] = evaluateFlagsFirst(lista)
/var/folders/k7/ncn3z0qs4h7f50pl2pw0l20r0000gp/T/ipykernel_22547/4214330145.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_final_copy['secondLavel'][i] = evaluateFlagsSecond(diccionario)


,nombre,subject_1-nota_y_p1,subject_1-nota_y_p2,subject_1-nota_y_p3,subject_1-nota_y_p4,subject_2-nota_p1,subject_2-nota_p2,subject_2-nota_p3,subject_2-nota_p4,subject_3-nota_p1,...,subject_10-nota_p3,subject_10-nota_p4,subject_11-nota_p1,subject_11-nota_p2,subject_11-nota_p3,subject_11-nota_p4,fecha,segment,firstLavel,secondLavel
647,MORALES RIOS EMANUEL,4.0,4.0,4.0,2.0,3.0,4.0,4.0,4.0,3.0,...,4.0,4.0,3.0,3.0,3.0,3.0,2021-03-01,TERCERO,GreenFlag,YellowFlag
650,MORENO MALAVER PAULINA,3.0,3.0,3.0,3.0,3.0,3.0,3.0,3.0,3.0,...,3.0,3.0,3.0,3.0,3.0,3.0,2021-03-01,SEGUNDO,GreenFlag,WhiteFlag
651,MORENO SALAZAR SARA YULIETH,2.0,4.0,4.0,4.0,2.0,2.0,2.0,4.0,4.0,...,3.0,4.0,4.0,3.0,3.0,4.0,2021-03-01,SEGUNDO,GreenFlag,WhiteFlag
652,MURCIA CARDENAS MANUEL JERONIMO,3.0,3.0,3.0,3.0,3.0,3.0,3.0,3.0,3.0,...,3.0,3.0,3.0,3.0,3.0,3.0,2021-03-01,TERCERO,GreenFlag,WhiteFlag
654,MURILLO BUITRAGO SUSANA,3.0,3.0,3.0,3.0,3.0,3.0,3.0,3.0,3.0,...,3.0,3.0,3.0,3.0,3.0,3.0,2021-03-01,CUARTO,GreenFlag,WhiteFlag
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
202,CARVAJAL GIL ISABELLA,4.0,4.0,4.0,4.0,4.0,4.0,3.0,3.0,3.0,...,3.0,3.0,3.0,3.0,4.0,4.0,2021-06-01,TERCERO,GreenFlag,WhiteFlag
204,CASALLAS LOPEZ NICOLAS,3.0,3.0,3.0,3.0,3.0,3.0,4.0,3.0,3.0,...,3.0,3.0,3.0,3.0,3.0,3.0,2021-06-01,QUINTO,GreenFlag,WhiteFlag
205,CASALLAS SALGADO MATHIAS,4.0,4.0,2.0,4.0,4.0,4.0,4.0,4.0,4.0,...,3.0,4.0,4.0,4.0,4.0,4.0,2021-06-01,SEGUNDO,GreenFlag,WhiteFlag
206,CASAS GARCIA SARA,4.0,4.0,2.0,4.0,4.0,4.0,4.0,4.0,4.0,...,3.0,4.0,4.0,4.0,3.0,4.0,2021-06-01,QUINTO,GreenFlag,YellowFlag


In [62]:
df_final_copy = df_final.head(1000).copy()

In [63]:
df_final_copy['firstLabel'] = 'NAN'
df_final_copy['secondLabel'] = 'NAN'
for i in df_final_copy.index:
    fecha = df_final_copy.fecha[i]
    nombre = df_final_copy.nombre[i]
    #first
    lista = flagsFirst(df_melt,nombre,fecha)
    df_final_copy['firstLabel'][i] = evaluateFlagsFirst(lista)
    #second
    diccionario = flagsSecond(df,fecha,nombre)
    df_final_copy['secondLabel'][i] = evaluateFlagsSecond(diccionario)
df_final_copy

ValueError: Can only compare identically-labeled Series objects

In [52]:
df_final_copy = df_final.iloc[500:1000].copy()
#df_final_copy

In [53]:
df_final_prueba = pd.DataFrame(columns=['nombre', 'fecha', 'firstLavel', 'secondLavel'])
for i in df_final_copy.index:
    fecha = df_final_copy.fecha[i]
    nombre = df_final_copy.nombre[i]
    #first
    lista = flagsFirst(df_melt,nombre,fecha)
    firstLavel = evaluateFlagsFirst(lista)
    #second
    diccionario = flagsSecond(df,fecha,nombre)
    secondLavel = evaluateFlagsSecond(diccionario)
    nueva_fila = {'nombre':nombre, 'fecha':fecha, 'firstLavel':firstLavel, 'secondLavel':secondLavel}
    df_final_prueba = df_final_prueba.append(nueva_fila, ignore_index=True)
df_final_prueba

,nombre,fecha,firstLavel,secondLavel
0,MORALES RIOS EMANUEL,2021-03-01,GreenFlag,YellowFlag
1,MORENO MALAVER PAULINA,2021-03-01,GreenFlag,WhiteFlag
2,MORENO SALAZAR SARA YULIETH,2021-03-01,GreenFlag,WhiteFlag
3,MURCIA CARDENAS MANUEL JERONIMO,2021-03-01,GreenFlag,WhiteFlag
4,MURILLO BUITRAGO SUSANA,2021-03-01,GreenFlag,WhiteFlag
...,...,...,...,...
495,CARVAJAL GIL ISABELLA,2021-06-01,GreenFlag,WhiteFlag
496,CASALLAS LOPEZ NICOLAS,2021-06-01,GreenFlag,WhiteFlag
497,CASALLAS SALGADO MATHIAS,2021-06-01,GreenFlag,WhiteFlag
498,CASAS GARCIA SARA,2021-06-01,GreenFlag,YellowFlag


In [ ]:
df_final_copy

In [58]:
df_final_prueba.iloc[498].nombre

'CASAS GARCIA SARA'